<a href="https://colab.research.google.com/github/luegl/keras-distinguish-own-cat-from-others-model/blob/main/predict_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download library

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

Import libraries

In [3]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Load and preprocess an image to make a prediction with the model

In [4]:
IMG_SIZE = (180, 180)
folder_path = "/content/drive/MyDrive/datasets/test"
yolo_model = YOLO("yolov8n.pt")
keras_model_path = "/content/drive/MyDrive/repos/keras-distinguish-own-cat-from-others-model/models/model1.keras"
keras_model = tf.keras.models.load_model(keras_model_path)

for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        img_path = os.path.join(folder_path, filename)

        img = Image.open(img_path)
        img_array = np.array(img)

        results = yolo_model.predict(source=img_path, conf=0.4, classes=[15])

        if results[0].boxes:
            boxes = results[0].boxes.xyxy.tolist()
            for box in boxes:
                x_min, y_min, x_max, y_max = map(int, box)

                cropped_img_array = img_array[y_min:y_max, x_min:x_max]
                if isinstance(cropped_img_array, tf.Tensor):
                    cropped_img_array = cropped_img_array.numpy()

                cropped_img = Image.fromarray(cropped_img_array.astype('uint8'))

                plt.figure(figsize=(10, 5))
                plt.subplot(1, 2, 1)
                plt.imshow(cropped_img)
                plt.axis("off")
                plt.title("Cropped Image")
                plt.subplot(1, 2, 2)
                plt.imshow(img)
                plt.axis("off")
                plt.title("Original Image")
                plt.show()

                cropped_img_resized = cropped_img.resize(IMG_SIZE)
                img_array_resized = keras.utils.img_to_array(cropped_img_resized)
                img_array_resized = img_array_resized / 255.0
                img_array_resized = tf.expand_dims(img_array_resized, axis=0)

                predictions = keras_model.predict(img_array_resized)

                score = float(predictions[0])
                print(f"Bild: {filename}")
                print(f"This image is {100 * (1 - score):.2f}% Flaekli and {100 * score:.2f}% other cat.")
                print("------------------------")
        else:
            print(f"No cat detected in {filename}")
            print("------------------------")

Output hidden; open in https://colab.research.google.com to view.